In [53]:
using HDF5

In [59]:
arxname="/home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5"

"/home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5"

In [60]:
filsss=h5open(arxname, "r")


HDF5 data file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5

In [61]:
HDF5.get_datasets(filsss)

7-element Array{HDF5Dataset,1}:
 HDF5 dataset: /CanalesSaturados (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )
 HDF5 dataset: /Canalesrespuesta (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )
 HDF5 dataset: /KTT_KInv (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )        
 HDF5 dataset: /LFPSaturados (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )    
 HDF5 dataset: /Nota (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )            
 HDF5 dataset: /freq (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )            
 HDF5 dataset: /kCSDCorrecta (file: /home/karel/RGutWork/JLDFiles/19115s1cut_single_event.h5xfer_mode: 0 )    

In [62]:
read(filsss["LFPSaturados"])

64×64×2000 Array{Float64,3}:
[:, :, 1] =
 -8.9152e304     -8.3914e148    …  0.0      0.0         18.1274 
  4.55151e256     1.09549e274      0.0      0.0          2.01416
 -2.4677e-70      3.51854e-179     0.0      0.0        -34.2407 
  9.50397e203    -3.54291e205      0.0      0.0      -4122.99   
 -1.93123e-5     -1.12043e-220     0.0      0.0         -2.01416
  2.24865e26    340.008         …  0.0      0.0      -4122.99   
  4.44929e200    -8.29891e288      0.0      0.0        -10.0708 
  1.23177e-104   -2.67882e303      0.0      0.0      -4122.99   
 -3.25855e23      3.03697e-240     0.0      0.0      -4122.99   
  1.01417e-161   -5.17802e253      0.0      0.0      -4122.99   
  1.35031e242    -1.79896e261   …  0.0      0.0      -4122.99   
 -1.19825e-162    3.82721e307      0.0      0.0         26.1841 
  1.47885e-269   -2.2847e161       0.0      0.0        -10.0708 
  ⋮                             ⋱                               
 -4.02873e299     1.37294e265      0.0  -4122.99 

In [64]:
h5read(arxname, "CanalesSaturados")

213-element Array{Int64,1}:
 11
  4
 13
  6
  2
 14
  8
  5
  1
 20
  6
  9
 23
  ⋮
  4
  1
 62
 64
  1
 59
 63
 61
 63
 62
 56
 63